# Import Data

In [12]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [13]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data Pre-processing

* Tokenization
* Remove Stopwords
* Remove words that have fewer than 3 chars
* Word lemmatization
* Word Stemming

In [14]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [15]:
np.random.seed(2018)

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joonhwan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizing Texts

In [17]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [20]:
# 도큐멘트 리스트 중 인덱스가 4310인 것으로 테스트
doc_sample = documents[documents['index'] == 4310].values[0][0]
print(doc_sample)

rain helps dampen bushfires


In [19]:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

original document: 
['rain', 'helps', 'dampen', 'bushfires']


In [21]:
print('tokenized and lemmatized document: ')
print(preprocess(doc_sample))

tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


### Check Lemmatizer

In [22]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

**- 여기까지는 data cleanup 과정**

# Bag of Words on the Dataset

* processed_doc 을 분석하여 유니크한 단어의 dictionary 를 만든다. 
* 이 과정을 통해 각각의 단어의 출현 빈도를 계산할 수 있다.

In [23]:
dictionary = gensim.corpora.Dictionary(processed_docs)

###  unique 한 단어 리스트

In [26]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
11 aust
12 rise
13 staff
14 strike
15 affect
16 australian
17 travel
18 ambiti
19 jump
20 olsson


In [25]:
print(dictionary)

Dictionary(62379 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)


#### 도큐멘트 내에서 단어의 출현빈도가 다음과 같은 경우 삭제 (filter out)
* 15개 이하의 도큐멘트에서 출현한 경우
* 0.5개 이상의 도큐멘트에서 출현한 경우
* 이중 100000개만 선택

In [28]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [31]:
len(dictionary) # 62379 -> 14174

14174

In [34]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4021, 1)]

In [38]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [91]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4021 ("dampen") appears 1 time.


# TF-IDF
* TF(단어 빈도, term frequency)는 특정한 단어가 문서에 얼마나 자주 등장하는지를 나타낸다. 예전에 다룬 빈도수 계산과 같다. 따라서 TF 값이 높을수로 문서 내에서 중요한 단어라고 간주된다.
* 문서군에서 자주 등장하는 단어의 빈도를 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다.
* TF-IDF는 TF와 IDF를 곱한 값으로 점수가 높은 단어일수록 전체 문서군에는 많이 등장하지 않지만, 특정 문서에서 자주 등장하는 단어를 의미한다. **즉 숫자가 높을수록 해당 문서 내에서 단어가 중요하다고 판단될 수 있다**

예: 
* doc1 = "I love icecream"
* doc2 = "I hate icecream and soda"
* doc3 = "Icecream is my favorite dessert during summer"

In [41]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903602896750699),
 (1, 0.38524510107363613),
 (2, 0.4974556071174764),
 (3, 0.5055678583740412)]


# Running LDA using Bag of Words

In [94]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [95]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"queensland" + 0.021*"countri" + 0.019*"hospit" + 0.018*"tasmanian" + 0.017*"hour" + 0.015*"work" + 0.015*"children" + 0.013*"student" + 0.013*"citi" + 0.011*"prison"
Topic: 1 
Words: 0.029*"australia" + 0.023*"world" + 0.021*"sydney" + 0.017*"market" + 0.012*"record" + 0.012*"share" + 0.011*"victoria" + 0.010*"industri" + 0.010*"australian" + 0.009*"port"
Topic: 2 
Words: 0.035*"trump" + 0.022*"kill" + 0.022*"north" + 0.020*"coast" + 0.016*"china" + 0.016*"attack" + 0.014*"adelaid" + 0.014*"west" + 0.013*"price" + 0.013*"gold"
Topic: 3 
Words: 0.027*"govern" + 0.016*"plan" + 0.014*"say" + 0.014*"rural" + 0.013*"council" + 0.012*"indigen" + 0.012*"turnbul" + 0.012*"water" + 0.011*"chang" + 0.011*"commun"
Topic: 4 
Words: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.014*"leagu" + 0.013*"lose" + 0.011*"john" + 0.010*"life" + 0.009*"premier" + 0.009*"unit" + 0.008*"compani"
Topic: 5 
Words: 0.027*"elect" + 0.023*"south" + 0.021*"nation" + 0.017*"final" + 0.01

In [15]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
 

def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


# create sample documents
doc_a = "I like to eat broccoli and bananas."
doc_b = "I ate a banana and spinach smoothie for breakfast."
doc_c = "Chinchillas and kittens are cute."
doc_d = "My sister adopted a kitten yesterday."
doc_e = "Look at this cute hamster munching on a piece of broccoli."
 
# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
 
# list for tokenized documents in loop
texts = []
 
# loop through document list
for i in doc_set:

    stemmed_tokens = preprocess(i)
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
 
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [16]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.119*"broccoli" + 0.119*"banana" + 0.071*"cute" + 0.071*"munch" + 0.071*"piec" + 0.071*"hamster" + 0.071*"look" + 0.071*"smoothi" + 0.071*"breakfast" + 0.071*"spinach"
Topic: 1 
Words: 0.166*"kitten" + 0.100*"cute" + 0.099*"yesterday" + 0.099*"adopt" + 0.099*"sister" + 0.099*"chinchilla" + 0.034*"like" + 0.034*"banana" + 0.034*"broccoli" + 0.034*"spinach"


In [18]:
test_document = 'Look at this cute hamster munching on a piece of broccoli.'
bow_vector = dictionary.doc2bow(preprocess(test_document))
for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 5)))

Score: 0.9181426167488098	 Topic: 0.119*"broccoli" + 0.119*"banana" + 0.071*"cute" + 0.071*"munch" + 0.071*"piec"
Score: 0.08185744285583496	 Topic: 0.166*"kitten" + 0.100*"cute" + 0.099*"yesterday" + 0.099*"adopt" + 0.099*"sister"


In [21]:
unseen_document = 'I used to eat brocolli for breakfast'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 5)))

Score: 0.7343907356262207	 Topic: 0.119*"broccoli" + 0.119*"banana" + 0.071*"cute" + 0.071*"munch" + 0.071*"piec"
Score: 0.2656092941761017	 Topic: 0.166*"kitten" + 0.100*"cute" + 0.099*"yesterday" + 0.099*"adopt" + 0.099*"sister"
